<a href="https://colab.research.google.com/github/pulindu-seniya-silva/FDM_data_preprocessing_task/blob/main/FDM_Data_preprocess_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load and quick scan

In [8]:
import pandas as pd
df = pd.read_csv('loan_prediction_dataset.csv')
df.head()
print(df.shape)
df.head()
print(df.dtypes)

(614, 13)
Loan_ID               object
Gender                object
Married               object
Dependents            object
Education             object
Self_Employed         object
ApplicantIncome        int64
CoapplicantIncome    float64
LoanAmount           float64
Loan_Amount_Term     float64
Credit_History       float64
Property_Area         object
Loan_Status           object
dtype: object


Missing-value auidt

In [9]:
missing = df.isna().sum().sort_values(ascending=False)
missing_pct = (missing/len(df)*100).round(2)
pd.DataFrame({"missing_count": missing, "missing_%": missing_pct})


,missing_count,missing_%
Credit_History,50,8.14
Self_Employed,32,5.21
LoanAmount,22,3.58
Dependents,15,2.44
Loan_Amount_Term,14,2.28
Gender,13,2.12
Married,3,0.49
Education,0,0.00
Loan_ID,0,0.00
CoapplicantIncome,0,0.00


Basic text cleanup (strings)

In [10]:
df_clean = df.copy()

# trim
for col in df_clean.select_dtypes(include="object").columns:
    df_clean[col] = df_clean[col].astype(str).str.strip()

# standardize placeholders -> NaN
df_clean.replace({"?": np.nan, "NA": np.nan, "None": np.nan, "nan": np.nan}, inplace=True)

# Example domain fix: Dependents '3+' -> 3 (numeric)
if "Dependents" in df_clean.columns:
    df_clean["Dependents"] = df_clean["Dependents"].replace({"3+": "3"})
    df_clean["Dependents"] = pd.to_numeric(df_clean["Dependents"], errors="coerce")
